In [2]:
from ultralytics import YOLO

In [5]:
#Enter path to weights
#Find models/weights in the google drive
model_path = "E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\runs\\detect\\negative_examplesv001\\weights\\best.pt"
model_path2 = "E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\cnn\\runs\\detect\\no_brightness_aug_\\weights\\best.pt"
mod_jouni = "E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\cnn\\runs\\detect\\masks_more_aug_more_train4\weights\\best.pt"
model = YOLO(model=model_path)

#Enter path for image to run model on
#ie detect clouds
#Works with different image sizes, however, the model was trained on 640x640 images
#(gör väl någon downsampling på större bilder, så förmodligen bäst att köra på bara 640*640?)
#Just nu sparar den bara bilden, kan användas för lite mer användbara saker också
#Find saved_chunks on the google drive
image_path = "E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\saved_chunks"
image2 = "E:\\scratch\\saved_chunks3"
results = model(image_path)
"""
https://docs.ultralytics.com/modes/predict
Results object consists of these component objects:

    Results.boxes: Boxes object with properties and methods for manipulating bboxes
    Results.masks: Masks object used to index masks or to get segment coordinates.
    Results.probs: torch.Tensor containing the class probabilities/logits.
    Results.orig_img: Original image loaded in memory.
    Results.path: Path containing the path to input image

Each result is composed of torch.Tensor by default, in which you can easily use following functionality:
results = results.cuda()
results = results.cpu()
results = results.to("cpu")
results = results.numpy()
"""



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/7087 E:\Kandidatarbete FITS\GitLab Repo\starseed\saved_chunks\00000_x_0_y_3000.png: 640x640 8 clouds, 14.0ms
image 2/7087 E:\Kandidatarbete FITS\GitLab Repo\starseed\saved_chunks\00001_x_320_y_3000.png: 640x640 5 clouds, 11.0ms
image 3/7087 E:\Kandidatarbete FITS\GitLab Repo\starseed\saved_chunks\00002_x_640_y_3000.png: 640x640 3 clouds, 11.0ms
image 4/7087 E:\Kandidatarbete FITS\GitLab Repo\starseed\saved_chunks\00003_x_960_y_3000.png: 640x640 1 cl

'\nhttps://docs.ultralytics.com/modes/predict\nResults object consists of these component objects:\n\n    Results.boxes: Boxes object with properties and methods for manipulating bboxes\n    Results.masks: Masks object used to index masks or to get segment coordinates.\n    Results.probs: torch.Tensor containing the class probabilities/logits.\n    Results.orig_img: Original image loaded in memory.\n    Results.path: Path containing the path to input image\n\nEach result is composed of torch.Tensor by default, in which you can easily use following functionality:\nresults = results.cuda()\nresults = results.cpu()\nresults = results.to("cpu")\nresults = results.numpy()\n'

In [6]:
import numpy as np
def calc_global_coord(res, reg_filename,bbox_list_name):
    results = res
    overlap = 320
    x_chunks = 373
    y_chunks = 19

    bbox_list = []
    bbox_with_conf = []
    x = -1
    y = 0
    for i in range(len(results)):
        image = results[i].boxes
        x += 1
        if x == int(x_chunks):
            x = 0
            y += 1
        for box in image:
            bbox_x = float(box.xywh[0][0])
            bbox_y = float(box.xywh[0][1])
            bbox_w = float(box.xywh[0][2])
            bbox_h = float(box.xywh[0][3])
            bbox_conf = float(box.conf)

            global_x = bbox_x + x * overlap
            global_y = bbox_y + y * overlap + 3000

            bbox_list.append((global_x, global_y, bbox_w, bbox_h))
            bbox_with_conf.append((global_x, global_y, bbox_w, bbox_h, bbox_conf))

    #Make sure to change path
    with open(f"E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\data\\regions\\{reg_filename}.reg", "w") as file:
        file.write("# Region file format: DS9 version 4.1\n")
        file.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')
        file.write("image\n")
        for box in bbox_list:
            x, y, w, h = box
            # Format: box(center_x, center_y, width, height)
            file.write("box({},{},{},{},0)\n".format(x, y, w, h))

    with open(f"E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\saved_chunks\\{bbox_list_name}.txt", "w") as file:
        for box in bbox_with_conf:
            x, y, w, h, conf = box
            # Format: center_x, center_y, width, height, confidence
            file.write("{},{},{},{},{}\n".format(x, y, w, h, conf))


In [7]:
calc_global_coord(results, "cnnrerun", "cnn_bboxes")

In [ ]:

import numpy as np
rel = "saved_chunks\bbox_with_conf010.txt"
abso = r"C:\Users\mull_\Desktop\Globala System\Kandidatarbete\Gitlab Repo\starseed-1\saved_chunks\bbox_with_conf010.txt"
def read_bbox_file(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        bbox_list = []
        for line in lines:
            x, y, w, h, conf = line.split(",")
            bbox_list.append((float(x), float(y), float(w), float(h), float(conf)))
    return bbox_list
bb_list = read_bbox_file(abso)


In [ ]:
#Sort bb_list by confidence
bb_list.sort(key=lambda x: x[4], reverse=True)

#Remove boxes with confidence > 0.2
bb_list1 = [box for box in bb_list if box[4] < 0.105]
for i in bb_list1:
    print(i)
print(len(bb_list1))

In [ ]:
# def list_to_region(lst):
with open(r"C:\Users\mull_\Desktop\Globala System\Kandidatarbete\Gitlab Repo\starseed-1\data\regions\tes105.reg", "w") as file:
    file.write("# Region file format: DS9 version 4.1\n")
    file.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')
    file.write("image\n")
    for box in bb_list1:
        x, y, w, h, conf = box
        # Format: box(center_x, center_y, width, height)
        file.write("box({},{},{},{},0)\n".format(x, y, w, h))
#list_to_region(bb_list1)

# Everything below here is just scratch code, don't run without triple checking. 
# Testing merge functions


In [ ]:
def iou(box1, box2):
    """Calculate the intersection over union (IOU) of two boxes."""
    x1, y1, w1, h1, conf1 = box1
    x2, y2, w2, h2, conf2 = box2
    inter_x = max(x1 - w1/2, x2 - w2/2)
    inter_y = max(y1 - h1/2, y2 - h2/2)
    inter_w = min(x1 + w1/2, x2 + w2/2) - inter_x
    inter_h = min(y1 + h1/2, y2 + h2/2) - inter_y
    inter_area = inter_w * inter_h
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area

def merge_boxes(boxes, iou_thresh=0.5):
    """Merge overlapping boxes using non-maximum suppression (NMS)."""
    boxes = np.array(boxes)
    box_scores = boxes[:, 4]
    sorted_idxs = np.argsort(box_scores)[::-1]
    merged_boxes = []
    while len(sorted_idxs) > 0:
        # Select the highest-scoring box that hasn't been merged yet
        curr_idx = sorted_idxs[0]
        curr_box = boxes[curr_idx]
        merged_boxes.append(curr_box)
        # Remove all boxes that have significant overlap with the current box
        overlap_idxs = np.where([iou(curr_box, boxes[i]) > iou_thresh for i in sorted_idxs])[0]
        sorted_idxs = np.delete(sorted_idxs, overlap_idxs)
    return merged_boxes

def nms(boxes, iou_threshold= 0.5):
    """
    Performs non-maximum suppression to remove overlapping boxes.
    
    Args:
        boxes (List[Tuple]): A list of tuples representing the boxes to be suppressed. Each
            tuple should contain the (center_x, center_y, width, height, score) values.
        iou_threshold (float): The IOU threshold to use when determining which boxes to suppress.
    
    Returns:
        List[Tuple]: A list of tuples representing the remaining boxes after suppression. Each
            tuple should contain the (center_x, center_y, width, height, score) values.
    """
    suppressed_boxes = []
    
    for i, box_a in enumerate(boxes):
        keep_box = True
        
        for j, box_b in enumerate(boxes):
            if i == j:
                continue
                
            iou2 = iou(box_a, box_b)
            
            if iou2 > iou_threshold:
                keep_box = False
                break
                
        if keep_box:
            suppressed_boxes.append(box_a)
    
    return suppressed_boxes


In [ ]:
#Read text file with bounding boxes and confidence scores
bbox_list = []
with open("E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\saved_chunks\\cnn_bbox.txt", "r") as file:
    for line in file:
        x, y, w, h, conf = line.split(",")
        bbox_list.append((float(x), float(y), float(w), float(h), float(conf)))



In [ ]:
supp_boxes = nms(bbox_list, iou_threshold= 0.5)

with open("E:\\Kandidatarbete FITS\\GitLab Repo\\starseed\\saved_chunks\\supp_boxes.txt", "w") as file:
    for box in supp_boxes:
        x, y, w, h, conf = box
        file.write("box({},{},{},{},0)\n".format(x, y, w, h))

In [ ]:

def NMS(boxes, overlapThresh=0.4):
    boxes = np.array(boxes)
    # Return an empty list, if no boxes given
    if len(boxes) == 0:
        return []
    # Compute the x and y coordinates of the top-left and bottom-right corners
    x1 = boxes[:, 0] - (boxes[:, 2] / 2)
    y1 = boxes[:, 1] + (boxes[:, 3] / 2)
    x2 = boxes[:, 0] + (boxes[:, 2] / 2)
    y2 = boxes[:, 1] - (boxes[:, 3] / 2)
    # Compute the area of the bounding boxes and sort the bounding
    # Boxes by the bottom-right y-coordinate of the bounding box
    areas = (x2 - x1 + 1) * (y2 - y1 + 1) # We add 1, because the pixel at the start as well as at the end counts
    # The indices of all boxes at start. We will redundant indices one by one.
    indices = np.arange(len(x1))
    for i, box in enumerate(boxes):
        # Create temporary indices  
        temp_indices = indices[indices!=i]
        # Find out the coordinates of the intersection box
        xx1 = np.maximum(x1[i], x1[temp_indices])
        yy1 = np.maximum(y1[i], y1[temp_indices])
        xx2 = np.minimum(x2[i], x2[temp_indices])
        yy2 = np.minimum(y2[i], y2[temp_indices])
        # Find out the width and the height of the intersection box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / areas[temp_indices]
        # if the actual boungding box has an overlap bigger than treshold with any other box, remove it's index  
        if np.any(overlap) > overlapThresh:
            indices = indices[indices != i]
    #return only the boxes at the remaining indices
    return boxes[indices].astype(int)

In [ ]:
bbox_arr = np.array(bbox_list)
sup_box = NMS(bbox_arr[:,:-1], overlapThresh=0.3)


In [ ]:
def merge_boxes(boxes, iou_threshold=0.5):
    merged_boxes = []
    # Iterate through each box
    for i, box in enumerate(boxes):
        # Check if the box has already been merged
        if i in merged_boxes:
            continue
        # Find all overlapping boxes
        overlapping_boxes = [j for j, b in enumerate(boxes) if j not in merged_boxes and iou(box, b) > iou_threshold]
        # Merge boxes and mark them as merged
        merged_box = box
        for j in overlapping_boxes:
            merged_box = merge(merged_box, boxes[j])
            merged_boxes.append(j)
        merged_boxes.append(i)
        # Add the merged box to the list of merged boxes
        merged_boxes.append(merged_box)
    return merged_boxes

def iou(box1, box2):
    # Calculate the intersection area
    x1 = max(box1[0]-box1[2]/2, box2[0]-box2[2]/2)
    y1 = max(box1[1]-box1[3]/2, box2[1]-box2[3]/2)
    x2 = min(box1[0]+box1[2]/2, box2[0]+box2[2]/2)
    y2 = min(box1[1]+box1[3]/2, box2[1]+box2[3]/2)
    intersection = max(0, x2-x1) * max(0, y2-y1)
    # Calculate the union area
    area1 = box1[2] * box1[3]
    area2 = box2[2] * box2[3]
    union = area1 + area2 - intersection
    # Calculate the IoU
    iou = intersection / union
    return iou

def merge(box1, box2):
    x = (box1[0] + box2[0]) / 2
    y = (box1[1] + box2[1]) / 2
    width = max(box1[0]+box1[2]/2, box2[0]+box2[2]/2) - min(box1[0]-box1[2]/2, box2[0]-box2[2]/2)
    height = max(box1[1]+box1[3]/2, box2[1]+box2[3]/2) - min(box1[1]-box1[3]/2, box2[1]-box2[3]/2)
    return [x, y, width, height]


In [8]:
from sklearn.cluster import DBSCAN
import numpy as np
import tqdm

def merge_boxes_progress(boxes, iou_threshold=0.5, epsilon=1000):
    merged_boxes = set()  # Changed to a set
    merged_boxes_list = []  # New list to keep track of merged boxes
    # Convert the boxes to a numpy array
    boxes_arr = np.array(boxes)
    # Cluster the boxes based on their x-coordinate
    db = DBSCAN(eps=epsilon, min_samples=2).fit(boxes_arr[:, [0]])
    labels = db.labels_
    unique_labels = set(labels)
    # Iterate through each cluster
    for label in unique_labels:
        if label == -1:
            continue
        cluster_boxes = boxes_arr[labels == label]
        # Sort the cluster by x-coordinate
        sorted_boxes = cluster_boxes[cluster_boxes[:, 0].argsort()]
        # Iterate through each box in the cluster
        for i, box in tqdm(enumerate(sorted_boxes), total=len(sorted_boxes), desc="Merging boxes"):
            # Check if the box has already been merged
            if i in merged_boxes:
                continue
            # Find all overlapping boxes within epsilon distance in the x direction
            overlapping_boxes = []
            for j, b in enumerate(sorted_boxes[i+1:], start=i+1):
                if abs(box[0] - b[0]) > epsilon:
                    break
                if j not in merged_boxes and iou(box, b) > iou_threshold:
                    overlapping_boxes.append(j)
            # Merge boxes and mark them as merged
            merged_box = box
            for j in overlapping_boxes:
                merged_box = merge(merged_box, sorted_boxes[j])
                merged_boxes.add(j)  # Add the index to the set
            merged_boxes.add(i)  # Add the index to the set
            merged_boxes_list.append(merged_box)  # Add the merged box to the list
    return merged_boxes_list


In [ ]:
new_bbox_list = [(box[0], box[1], box[2], box[3]) for box in bbox_list]

test_merges = merge_boxes_progress(new_bbox_list, iou_threshold=0.5)


In [ ]:
def merge_boxes_until_converged(boxes, iou_threshold=0.5, epsilon=1000, delta=1, max_iter = 3):
    """
    Calls merge_boxes_progress until the output from the previous run is only delta elements longer.
    """
    current_boxes = boxes
    previous_len = len(current_boxes)
    iteration = 0
    while True:
        iteration += 1
        if iteration > max_iter:
            break
        merged_boxes = merge_boxes_progress(current_boxes, iou_threshold=iou_threshold, epsilon=epsilon)
        if previous_len - len(merged_boxes) <= delta:
            break
        current_boxes = merged_boxes
        previous_len = len(current_boxes)
        print(previous_len)

    return current_boxes

test4 = merge_boxes_until_converged(test3, iou_threshold=0.4, epsilon=1000, delta=1)
print(len(test4))
